## EDA understanding of your dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../data/taxi_trip_pricing.csv")
df.head()

In [ ]:
df.info()


In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

## Choices of my features and label for the ML model

In [ ]:
selected_cols = [
    "Trip_Distance_km",
    "Trip_Duration_Minutes",
    "Time_of_Day",
    "Passenger_Count",
    "Trip_Price"

]

df_cleaned = df[selected_cols]

## EDA on the new data

In [ ]:
df_cleaned.hist(figsize=(10,8))
plt.show()